In [2]:
import numpy as np
import math
from enum import Enum
from collections import deque
from bisect import insort
from pyvis.network import Network


In [60]:
class Action(Enum):
    UP = 0
    RIGHT = 1
    DOWN = 2
    LEFT = 3

    def __str__(self):
        if self == Action.UP:
            return 'UP'

        if self == Action.RIGHT:
            return 'RIGHT'

        if self == Action.DOWN:
            return 'DOWN'

        if self == Action.LEFT:
            return 'LEFT'


N = 3
BLANK = N*N
#initial_state = [1, 2, BLANK, 4, 5, 3, 7, 8, 6]
#initial_state = [BLANK, 2, 3 ,4 ,5 ,6,7,1,8]
#intial_state = [1,2,3,4,5,6,BLANK,7,8]
initial_state = [4, 6, 1, 3, 2, BLANK, 5, 7, 8]
#initial_state = [1,2,3,4,5,6,7,BLANK,8]
goal_state = list(map(lambda x: x, range(1, N*N+1)))


TODO: aclarar condición de solucionable


In [4]:
def count_inversions(state, n):
    count = 0

    for i in range(0, n):
        for j in range(i+1, n):
            if state[j] != BLANK and state[i] != BLANK and state[i] > state[j]:
                count += 1

    return count


def is_solvable(initial_state, n):
    inversions = count_inversions(initial_state, n)

    if n % 2 == 1:
        return inversions % 2 == 0
    else:
        blank_index = find_blank_index(initial_state)
        row_from_bottom = N - get_tile_row(blank_index, n)

        if row_from_bottom % 2 == 0:
            return inversions % 2 == 1
        else:
            return inversions % 2 == 0


def get_next_state(state, action, n):
    new_state = state.copy()
    blank_index = find_blank_index(state)

    if action == Action.UP:
        new_state[blank_index], new_state[blank_index -
                                          n] = new_state[blank_index-n], new_state[blank_index]
    elif action == Action.RIGHT:
        new_state[blank_index], new_state[blank_index +
                                          1] = new_state[blank_index+1], new_state[blank_index]
    elif action == Action.DOWN:
        new_state[blank_index], new_state[blank_index +
                                          n] = new_state[blank_index+n], new_state[blank_index]
    elif action == Action.LEFT:
        new_state[blank_index], new_state[blank_index -
                                          1] = new_state[blank_index-1], new_state[blank_index]

    return new_state


def get_actions(state, n):
    blank_index = find_blank_index(state)
    blank_col = blank_index % n
    actions = []

    if blank_col > 0:
        actions.append(Action.LEFT)
    if blank_index <= n*(n - 1) - 1:
        actions.append(Action.DOWN)
    if blank_col < n-1:
        actions.append(Action.RIGHT)
    if blank_index >= n:
        actions.append(Action.UP)

    return actions


def is_goal_state(state):
    return state == goal_state


def is_blank(tile):
    return tile == BLANK


def find_blank_index(state):
    return state.index(BLANK)


def get_tile_col(tile_index, n):
    return tile_index % n


def get_tile_row(tile_index, n):
    return math.floor(tile_index / n)


In [71]:
class Node:
    id = -1

    def __init__(self, state, parent, action, path_cost, depth, estimated_cost=0):
        Node.id = Node.id+1
        self.id = Node.id

        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.estimated_cost = estimated_cost
        self.depth = depth

    def __lt__(self, node):
        return self.estimated_cost < node.estimated_cost


    def __str__(self):
        if self.parent is not None:
            return f"State:{self.state}\tParent State:{self.parent.state}\tAction:{self.action}\tPath Cost:{self.path_cost}\tEstimated Cost:{self.estimated_cost}\n"
        return f"State:{self.state}\tParent State:{self.parent}\tAction:{self.action}\tPath Cost:{self.path_cost}\tEstimated Cost:{self.estimated_cost}\n"


TODO: agregar funciones para estadísticas


In [ ]:
def DFS(initial_state, n):
    root = Node(initial_state, None, None, 0, 0)
    frontier = deque([root])
    visited_states = {}
    visited_states[tuple(initial_state)] = True
    visited_node_count = 1

    tree_vis = Network(notebook=True, layout='hierarchical')
    frontier_len = len(frontier)
    tree_vis.add_node(root.id, label=str(tuple(root.state)),
                      level=root.depth, color='#dd4b39')
    while frontier_len > 0:
        node = frontier.popleft()
        visited_states[tuple(node.state)] = True
        visited_node_count += 1
        frontier_len -= 1
        if is_goal_state(node.state):
            tree_vis.prep_notebook()
            tree_vis.show('example.html')
            return node
        else:
            actions = get_actions(node.state, n)
            for action in actions:
                new_state = get_next_state(node.state, action, n)
                new_node = Node(new_state, node, action,
                                node.path_cost + 1, node.depth+1)
                tree_vis.add_node(new_node.id, label=str(
                    tuple(new_node.state)), level=new_node.depth)
                tree_vis.add_edge(new_node.id, new_node.parent.id,
                                  label=str(new_node.action))
                if not visited_states.get(tuple(new_state)):
                    frontier.appendleft(new_node)
                    frontier_len += 1


In [ ]:
if is_solvable(initial_state, N):
    ans = DFS(initial_state, N)
    current = ans
    while current is not None:
        print(current)
        current = current.parent

else:
    print("unsolvable initial state")


In [5]:
def BFS(initial_state, n):
    root = Node(initial_state, None, None, 0, 0)
    frontier = deque([root])
    visited_states = {}
    visited_states[tuple(initial_state)] = True
    visited_node_count = 1

    tree_vis = Network(notebook=True, layout='hierarchical')
    frontier_len = len(frontier)
    tree_vis.add_node(root.id, label=str(tuple(root.state)),
                      level=root.depth, color='#dd4b39')
    while frontier_len > 0:
        node = frontier.popleft()
        visited_states[tuple(node.state)] = True
        visited_node_count += 1
        frontier_len -= 1
        if is_goal_state(node.state):
            tree_vis.prep_notebook()
            tree_vis.show('example.html')
            return node
        else:
            actions = get_actions(node.state, n)
            for action in actions:
                new_state = get_next_state(node.state, action, n)
                new_node = Node(new_state, node, action,
                                node.path_cost + 1, node.depth+1)
                tree_vis.add_node(new_node.id, label=str(
                    tuple(new_node.state)), level=new_node.depth)
                tree_vis.add_edge(new_node.id, new_node.parent.id,
                                  label=str(new_node.action))
                if not visited_states.get(tuple(new_state)):
                    frontier.append(new_node)
                    frontier_len += 1


In [6]:
if is_solvable(initial_state, N):
    ans = BFS(initial_state, N)
    current = ans
    while current is not None:
        print(current)
        current = current.parent

else:
    print("unsolvable initial state")


unsolvable initial state


In [ ]:
def DLS(intial_state, n, max_depth):
    root = Node(initial_state, None, None, 0, 0)
    frontier = deque([root])
    visited_states = {}
    visited_states[tuple(initial_state)] = 0
    visited_node_count = 1
    tree_vis = Network(notebook=True, layout='hierarchical')
    frontier_len = len(frontier)
    tree_vis.add_node(root.id, label=str(tuple(root.state)),
                      level=root.depth, color='#dd4b39')

    while frontier_len > 0:
        node = frontier.popleft()
        visited_states[tuple(node.state)] = node.depth
        visited_node_count += 1
        frontier_len -= 1
        if node.depth > max_depth:
            continue
        elif is_goal_state(node.state):
            tree_vis.prep_notebook()
            tree_vis.show('example.html')
            return node

        else:
            actions = get_actions(node.state, n)

            for action in actions:
                new_state = get_next_state(node.state, action, n)
                new_node = Node(new_state, node, action,
                                node.path_cost + 1, node.depth+1)
                tree_vis.add_node(new_node.id, label=str(
                    tuple(new_node.state)), level=new_node.depth)
                tree_vis.add_edge(new_node.id, new_node.parent.id,
                                  label=str(new_node.action))

                if visited_states.get(tuple(new_state)) is None or visited_states.get(tuple(new_state)) > new_node.depth:
                    frontier.appendleft(new_node)
                    frontier_len += 1

    tree_vis.prep_notebook()
    tree_vis.show('example.html')


In [ ]:
if is_solvable(initial_state, N):
    ans = DLS(initial_state, N, 15)
    current = ans
    while current is not None:
        print(current)
        current = current.parent

else:
    print("unsolvable initial state")


In [ ]:
def IDS(initial_state, n, initial_limit):
    limit = initial_limit
    result = DLS(initial_state, n, limit)
    if result is None:
        while result is None:
            limit += 1
            result = DLS(initial_state, n, limit)
    else:
        lower_limit = 0
        upper_limit = result.depth
        while lower_limit <= upper_limit:
            mid = lower_limit + (upper_limit - lower_limit) // 2
            result = DLS(initial_state, n, mid)

            if result is None:
                lower_limit = mid + 1
            else:
                last_result = result
                upper_limit = mid - 1

    return result


In [ ]:
if is_solvable(initial_state, N):
    ans = IDS(initial_state, N, 10)
    current = ans
    while current is not None:
        print(current)
        current = current.parent

else:
    print("unsolvable initial state")


In [64]:
def LHS(initial_state, n, heuristic):
    root = Node(initial_state, None, None, 0, 0)
    frontier = deque([root])
    visited_node_count = 1

    visited_states = {}
    visited_states[tuple(initial_state)] = True

    tree_vis = Network(notebook=True, layout='hierarchical')
    frontier_len = len(frontier)
    tree_vis.add_node(root.id, label=str(tuple(root.state)),
                      level=root.depth, color='#dd4b39')

    while frontier_len > 0:
        node = frontier.popleft()
        visited_states[tuple(node.state)] = True
        visited_node_count += 1
        frontier_len -= 1

        if is_goal_state(node.state):
            tree_vis.prep_notebook()
            tree_vis.show('example.html')
            return node
        else:
            actions = get_actions(node.state, n)
            successors = []

            for action in actions:
                new_state = get_next_state(node.state, action, n)
                new_node = Node(new_state, node, action,
                                node.path_cost + 1, node.depth+1, heuristic(new_state, n))
                tree_vis.add_node(new_node.id, label="Estimated cost: " +str(new_node.estimated_cost) + "\n" + str(
                    tuple(new_node.state)), level=new_node.depth)
                tree_vis.add_edge(new_node.id, new_node.parent.id,
                                  label=str(new_node.action))
                if not visited_states.get(tuple(new_state)):
                    # insort(successors, new_node,*[n.estimated_cost for n in successors])
                    # insort(successors, new_node, *["estimated_cost"])
                    successors.append(new_node)
                    frontier_len += 1
            successors.sort(key=lambda n : n.estimated_cost, reverse=True)
            frontier.extendleft(successors)


In [29]:
def manhattan_distance(tile_index, tile, n):
    if tile_index == tile - 1 or tile == BLANK:
        return 0

    tile_row = get_tile_row(tile_index, n)
    tile_col = get_tile_col(tile_index, n)

    tile_row_goal_position = get_tile_row(tile - 1, n)
    tile_col_goal_position = get_tile_col(tile - 1, n);

    distance = abs(tile_row - tile_row_goal_position) + abs(tile_col - tile_col_goal_position)
    return distance

def euclidean_distance(tile_index, tile, n):
    if tile_index == tile - 1 or tile == BLANK:
        return 0

    tile_row = get_tile_row(tile_index, n)
    tile_col = get_tile_col(tile_index, n)

    tile_row_goal_position = get_tile_row(tile - 1, n)
    tile_col_goal_position = get_tile_col(tile - 1, n);

    row_distance = abs(tile_row - tile_row_goal_position)
    col_distance = abs(tile_col - tile_col_goal_position)

    distance = math.sqrt(row_distance*row_distance + col_distance*col_distance)
    return distance

def tile_in_position(tile_index, tile, n): 
    if tile_index == tile - 1 or tile == BLANK:
        return 0
    return 1
    

In [65]:
def heuristic(state, n):
    sum = 0
    for tile_index, tile in enumerate(state):
        sum += tile_in_position(tile_index, tile, n)
    return sum
   


if is_solvable(initial_state, N):
    ans = LHS(initial_state, N, heuristic)
    current = ans
    while current is not None:
        print(current)
        current = current.parent

else:
    print("unsolvable initial state")

State:[1, 2, 3, 4, 5, 6, 7, 8, 9]	Parent State:[1, 2, 3, 4, 5, 9, 7, 8, 6]	Action:DOWN	Path Cost:1847	Estimated Cost:0

State:[1, 2, 3, 4, 5, 9, 7, 8, 6]	Parent State:[1, 2, 3, 4, 9, 5, 7, 8, 6]	Action:RIGHT	Path Cost:1846	Estimated Cost:1

State:[1, 2, 3, 4, 9, 5, 7, 8, 6]	Parent State:[1, 9, 3, 4, 2, 5, 7, 8, 6]	Action:DOWN	Path Cost:1845	Estimated Cost:2

State:[1, 9, 3, 4, 2, 5, 7, 8, 6]	Parent State:[1, 3, 9, 4, 2, 5, 7, 8, 6]	Action:LEFT	Path Cost:1844	Estimated Cost:3

State:[1, 3, 9, 4, 2, 5, 7, 8, 6]	Parent State:[1, 3, 5, 4, 2, 9, 7, 8, 6]	Action:UP	Path Cost:1843	Estimated Cost:4

State:[1, 3, 5, 4, 2, 9, 7, 8, 6]	Parent State:[1, 3, 5, 4, 2, 6, 7, 8, 9]	Action:UP	Path Cost:1842	Estimated Cost:4

State:[1, 3, 5, 4, 2, 6, 7, 8, 9]	Parent State:[1, 3, 5, 4, 2, 6, 7, 9, 8]	Action:RIGHT	Path Cost:1841	Estimated Cost:3

State:[1, 3, 5, 4, 2, 6, 7, 9, 8]	Parent State:[1, 3, 5, 4, 9, 6, 7, 2, 8]	Action:DOWN	Path Cost:1840	Estimated Cost:4

State:[1, 3, 5, 4, 9, 6, 7, 2, 8]	Parent S

In [67]:
def GHS(initial_state, n, heuristic):
    root = Node(initial_state, None, None, 0, 0)
    frontier = deque([(0, root)])
    visited_states = {}
    visited_states[tuple(initial_state)] = True
    visited_node_count = 1

    tree_vis = Network(notebook=True, layout='hierarchical')
    frontier_len = len(frontier)
    tree_vis.add_node(root.id, label=str(tuple(root.state)),
                      level=root.depth, color='#dd4b39')
    while frontier_len > 0:
        node = frontier.popleft()[1]
        visited_states[tuple(node.state)] = True
        visited_node_count += 1
        frontier_len -= 1

        if is_goal_state(node.state):
            tree_vis.prep_notebook()
            tree_vis.show('example.html')
            return node
        else:
            actions = get_actions(node.state, n)
            
            for action in actions:
                new_state = get_next_state(node.state, action, n)
                new_node = Node(new_state, node, action,
                                node.path_cost + 1, node.depth+1, heuristic(new_state, n))
                tree_vis.add_node(new_node.id, label="Estimated cost: " +str(new_node.estimated_cost) + "\n" + str(
                    tuple(new_node.state)), level=new_node.depth)
                tree_vis.add_edge(new_node.id, new_node.parent.id,
                                  label=str(new_node.action))
                                  
                if not visited_states.get(tuple(new_state)):
                    insort(frontier, new_node)
                    frontier_len += 1
                
        

In [72]:
def heuristic(state, n):
    sum = 0
    for tile_index, tile in enumerate(state):
        sum += manhattan_distance(tile_index, tile, n)
    return sum
   


if is_solvable(initial_state, N):
    ans = GHS(initial_state, N, heuristic)
    current = ans
    while current is not None:
        print(current)
        current = current.parent

else:
    print("unsolvable initial state")

State:[1, 2, 3, 4, 5, 6, 7, 8, 9]	Parent State:[1, 2, 3, 4, 5, 6, 7, 9, 8]	Action:RIGHT	Path Cost:25	Estimated Cost:0

State:[1, 2, 3, 4, 5, 6, 7, 9, 8]	Parent State:[1, 2, 3, 4, 9, 6, 7, 5, 8]	Action:DOWN	Path Cost:24	Estimated Cost:1

State:[1, 2, 3, 4, 9, 6, 7, 5, 8]	Parent State:[1, 9, 3, 4, 2, 6, 7, 5, 8]	Action:DOWN	Path Cost:23	Estimated Cost:2

State:[1, 9, 3, 4, 2, 6, 7, 5, 8]	Parent State:[1, 3, 9, 4, 2, 6, 7, 5, 8]	Action:LEFT	Path Cost:22	Estimated Cost:3

State:[1, 3, 9, 4, 2, 6, 7, 5, 8]	Parent State:[1, 3, 6, 4, 2, 9, 7, 5, 8]	Action:UP	Path Cost:21	Estimated Cost:4

State:[1, 3, 6, 4, 2, 9, 7, 5, 8]	Parent State:[1, 3, 6, 4, 9, 2, 7, 5, 8]	Action:RIGHT	Path Cost:20	Estimated Cost:5

State:[1, 3, 6, 4, 9, 2, 7, 5, 8]	Parent State:[1, 3, 6, 4, 5, 2, 7, 9, 8]	Action:UP	Path Cost:19	Estimated Cost:6

State:[1, 3, 6, 4, 5, 2, 7, 9, 8]	Parent State:[1, 3, 6, 4, 5, 2, 7, 8, 9]	Action:LEFT	Path Cost:18	Estimated Cost:5

State:[1, 3, 6, 4, 5, 2, 7, 8, 9]	Parent State:[1, 3, 6, 4